<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/JavaScripts/ImageCollection/SimpleCloudScore.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/JavaScripts/ImageCollection/SimpleCloudScore.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=JavaScripts/ImageCollection/SimpleCloudScore.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/JavaScripts/ImageCollection/SimpleCloudScore.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [ ]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

Import libraries

In [ ]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. 

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [ ]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [ ]:
# SimpleCloudScore, an example of computing a cloud-free composite with L8
# by selecting the least-cloudy pixel from the collection.

# A mapping from a common name to the sensor-specific bands.
LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6',    'B7',    'B10']
STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'temp']

# Compute a cloud score.  This expects the input image to have the common
# band names: ["red", "blue", etc], so it can work across sensors.
def cloudScore(img):
  # A helper to apply an expression and linearly rescale the output.
  def rescale(img, exp, thresholds):
    return img.expression(exp, {'img': img}) \
        .subtract(thresholds[0]).divide(thresholds[1] - thresholds[0])
  

  # Compute several indicators of cloudyness and take the minimum of them.
  score = ee.Image(1.0)
  # Clouds are reasonably bright in the blue band.
  score = score.min(rescale(img, 'img.blue', [0.1, 0.3]))

  # Clouds are reasonably bright in all visible bands.
  score = score.min(rescale(img, 'img.red + img.green + img.blue', [0.2, 0.8]))

  # Clouds are reasonably bright in all infrared bands.
  score = score.min(
      rescale(img, 'img.nir + img.swir1 + img.swir2', [0.3, 0.8]))

  # Clouds are reasonably cool in temperature.
  score = score.min(rescale(img, 'img.temp', [300, 290]))

  # However, clouds are not snow.
  ndsi = img.normalizedDifference(['green', 'swir1'])
  return score.min(rescale(ndsi, 'img', [0.8, 0.6]))


# Filter the TOA collection to a time-range and add the cloudscore band.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterDate('2017-05-01', '2017-07-01')

def func_wwa(img):
      # Invert the cloudscore so 1 is least cloudy, and rename the band.
      score = cloudScore(img.select(LC8_BANDS, STD_NAMES))
      score = ee.Image(1).subtract(score).select([0], ['cloudscore'])
      return img.addBands(score) \
    .map(func_wwa)







# Define visualization parameters for a True color image.
vizParams = {'bands': ['B4',  'B3',  'B2'], 'max': 0.4, 'gamma': 1.6}
Map.setCenter(-120.24487, 37.52280, 8)
Map.addLayer(collection.qualityMosaic('cloudscore'), vizParams)


## Display Earth Engine data layers 

In [ ]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map